In [1]:
import praw
import math
import pandas as pd
from datetime import datetime
import re
import time
from tqdm import tqdm
tqdm.pandas()

user_agent="agent01"
reddit = praw.Reddit(username="No-Professional-8030",
                     password="Tutorial01",
                     client_id="2MS-Ef8sTpSYzdbqEpIxAg",
                     client_secret="Cnht4sjg8xG0EZTaPI8Yk0Kn3zGKqw",
                     user_agent=user_agent
)
subreddit_name="buildapcsales"
subreddit = reddit.subreddit(subreddit_name)
print(subreddit.display_name)

hot_subreddit=subreddit.hot()

df = pd.DataFrame()
post=[]
type = ''
price = 0.00
item = ''
regex= r"\[([^\]]+)\](.+?)(\$\d+)"
for submission in subreddit.hot():
    if(submission.id in ('yvocv1','z2atkk')):
        continue
    result = re.search(r'\[([^\]]+)\](.+?)(\$\d+\.?\d+)', submission.title)
    type=result.group(1)
    item=result.group(2).lstrip()
    price=result.group(3)
#     print(type)
#     print(item)
#     print(price)
    post.append([submission.id,type,item,price,submission.score,submission.url,submission.created])

post = pd.DataFrame(post,columns=['id','Type','Item','Price','score','url','Time Created'])

def get_comments(submission_id):
    submission = reddit.submission(submission_id)
    comments = []
    for top_level_comment in submission.comments:
        comments.append(top_level_comment.body)
        
    comment = ' '.join(comments)
    comment = comment.replace("\n",' ')
    return comment


def sentiment_scores(sentence):

    sid_obj = SentimentIntensityAnalyzer()
    sentiment_dict = sid_obj.polarity_scores(sentence)
    
    if sentiment_dict['compound'] >= 0.05 :
        return "Positive"
    elif sentiment_dict['compound'] <= - 0.05 :
        return "Negative"
    else :
        return "Neutral"
    
    
def get_date_time(x):
    return datetime.fromtimestamp(x)#.strftime('%Y-%m-%d')

def get_post_hours(x):
    return (time.time() - x)/3600

def search_item(x):
    return any([True if i.lower() in x.lower() else False for i in item.split()])


post['Comment'] = post['id'].progress_apply(lambda x: get_comments(x))

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

post['Sentiment'] = post['Comment'].progress_apply(lambda x: sentiment_scores(x))


post['Time Created1'] = post['Time Created'].progress_apply(lambda x: get_date_time(x))


post['LikesPerHour'] = post.progress_apply(lambda x: math.ceil(x['score']/get_post_hours(x['Time Created'])),axis=1)

post.rename(columns={'score':'Overall Likes'},inplace=True)

post.drop(['Time Created'],axis=1,inplace=True)

post.rename(columns={'Time Created1':'Post Datetime'},inplace=True)

n = int(input("How many likes per hour ratio does the post have: "))
item = input("What type of sale item are they looking for: ")

post_filtered = post[post['LikesPerHour']>=n]

post_filtered = post_filtered[post_filtered['Type'].apply(lambda name: search_item(name))]

post_filtered.to_csv('post.csv',index=False) # use this post.csv to load into google sheet

Version 7.6.0 of praw is outdated. Version 7.6.1 was released Friday November 11, 2022.


buildapcsales


100%|███████████████████████████████████████████████████████████████████████████████| 98/98 [02:29<00:00,  1.53s/it]
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\avitr\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
100%|████████████████████████████████████████████████████████████████████████████| 98/98 [00:00<00:00, 13947.33it/s]


How many likes per hour ratio does the post have: 1
What type of sale item are they looking for: ssd


In [3]:
post

,id,Type,Item,Price,Overall Likes,url,Comment,Sentiment,Post Datetime,LikesPerHour
0,zsjx3n,HDMI Cable,"Amazon Basics 15 Feet HDMI Cable (18Gbps, 4K/6...",$3.47,176,https://www.amazon.com/dp/B07SBKPYK6,Be mindful of recent listings of in-demand pro...,Positive,2022-12-22 16:58:02,31
1,zsmpzs,HDD,Seagate 18TB Exos X18 7200 RPM SATA,$269,36,https://www.newegg.com/seagate-exos-x18-st1800...,Be mindful of recent listings of in-demand pro...,Neutral,2022-12-22 19:30:09,12
2,zsmmrs,Accessories,PlayStation DualSense Wireless Controller - Co...,$49,37,https://www.newegg.com/p/N82E16879261900?Item=...,Be mindful of recent listings of in-demand pro...,Positive,2022-12-22 19:25:52,12
3,zsbpw7,SSD - M.2,Crucial P5 Plus 2TB Internal SSD PCIe Gen 4 x4...,$149.99,180,https://www.bestbuy.com/site/crucial-p5-plus-2...,My 2TB XPG S70 Blade(boot drive) unfortunately...,Positive,2022-12-22 09:38:39,14
4,zspmd2,Other,Elgato - Wave XLR - XLR/USB-C Microphone Inter...,$119.99,13,https://www.bestbuy.com/site/elgato-wave-xlr-x...,"I have this, and it works pretty well, softwar...",Positive,2022-12-22 21:36:05,12
...,...,...,...,...,...,...,...,...,...,...
93,zpepa1,SSD,Sabrent 1TB Rocket Q4 NVMe PCIe 4.0 M.2 2280 I...,$82.99,68,https://computers.woot.com/offers/sabrent-1tb-...,I can’t comment on this drive at all. But I lo...,Positive,2022-12-19 07:23:58,1
94,zpk7lr,SSD,Nextorage NEM-PA series 1TB m.2 PCIe gen4 NVMe...,$87.99,28,https://www.newegg.com/nextorage-1tb-nem-pa-se...,Be mindful of recent listings of in-demand pro...,Positive,2022-12-19 12:04:00,1
95,zpl1a8,Case,NZXT H7 Mid Tower Case (Black/White),$69.99,21,https://www.amazon.com/dp/B09VNJZ8Z1?psc=1,Be mindful of recent listings of in-demand pro...,Neutral,2022-12-19 12:52:21,1
96,zpep69,OTHER,SteelSeries QcK Gaming Surface XL RGB Prism Cl...,$34.99,45,https://www.amazon.com/gp/product/B07HYXKWS9/r...,Be mindful of recent listings of in-demand pro...,Positive,2022-12-19 07:23:49,1
